In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectFromModel

In [2]:
df = pd.read_csv("preprocessed_data.csv")
df.head()

,rooms,floor,floorCount,latitude,longitude,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,...,city_lodz,city_lublin,city_poznan,city_radom,city_rzeszow,city_szczecin,city_warszawa,city_wroclaw,ownership_cooperative,ownership_udział
0,4.0,3.0,4.0,1.094088,-2.820267,-0.816073,1.487393,-0.026132,0.230208,1.028105,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2.0,5.0,5.0,1.092505,-2.781375,1.647109,-0.585392,-0.607703,-0.383685,-0.380457,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,5.0,4.0,4.0,1.102938,-2.777586,-0.440334,-0.432099,-0.410059,0.479928,-0.185681,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,3.0,7.0,7.0,1.106274,-2.779584,-0.440334,-0.316574,-0.228318,0.340501,-0.233188,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3.0,4.0,4.0,1.112952,-2.765224,-0.565580,-0.603165,0.413455,0.860750,1.358273,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
X = df.drop(columns=["log_price"])
y = df["log_price"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Tuning

In [5]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 500, 2000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "max_depth": trial.suggest_int("max_depth", 5, 15),
        "num_leaves": trial.suggest_int("num_leaves", 20, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0)
    }
    model = lgb.LGBMRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)
best_params = study.best_params

[I 2025-03-13 22:35:28,262] A new study created in memory with name: no-name-0908b654-c82e-4510-b2d7-ef863954ed59


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016847 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3347
[LightGBM] [Info] Number of data points in the train set: 33385, number of used features: 38
[LightGBM] [Info] Start training from score 13.507787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2025-03-13 22:35:33,419] Trial 0 finished with value: 0.07940447102127102 and parameters: {'n_estimators': 900, 'learning_rate': 0.09438794747742887, 'max_depth': 9, 'num_leaves': 35, 'subsample': 0.9350265479714479, 'colsample_bytree': 0.6017162763709837}. Best is trial 0 with value: 0.07940447102127102.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009896 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3347
[LightGBM] [Info] Number of data points in the train set: 33385, number of used features: 38
[LightGBM] [Info] Start training from score 13.507787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[I 2025-03-13 22:35:39,912] Trial 1 finished with value: 0.08863343352236716 and parameters: {'n_estimators': 1400, 'learning_rate': 0.027346736903409136, 'max_depth': 8, 'num_leaves': 34, 'subsample': 0.5051200082051728, 'colsample_bytree': 0.8769088278940664}. Best is trial 0 with value: 0.07940447102127102.


...

[I 2025-03-13 22:37:19,754] Trial 17 finished with value: 0.07238668511534535 and parameters: {'n_estimators': 1100, 'learning_rate': 0.047085332891749, 'max_depth': 14, 'num_leaves': 79, 'subsample': 0.7572874138151058, 'colsample_bytree': 0.7673598285325841}. Best is trial 15 with value: 0.0573831543762593.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005195 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3347
[LightGBM] [Info] Number of data points in the train set: 33385, number of used features: 38
[LightGBM] [Info] Start training from score 13.507787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2025-03-13 22:37:27,071] Trial 18 finished with value: 0.06163943245293069 and parameters: {'n_estimators': 1900, 'learning_rate': 0.08836356396316501, 'max_depth': 13, 'num_leaves': 57, 'subsample': 0.6692402271173643, 'colsample_bytree': 0.918935157563197}. Best is trial 15 with value: 0.0573831543762593.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004983 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3347
[LightGBM] [Info] Number of data points in the train set: 33385, number of used features: 38
[LightGBM] [Info] Start training from score 13.507787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[I 2025-03-13 22:37:35,282] Trial 19 finished with value: 0.05769306098972569 and parameters: {'n_estimators': 1800, 'learning_rate': 0.09897453519998078, 'max_depth': 11, 'num_leaves': 81, 'subsample': 0.5930156850525687, 'colsample_bytree': 0.7949028743508253}. Best is trial 15 with value: 0.0573831543762593.


In [6]:
lgb_model = lgb.LGBMRegressor(**best_params, random_state=42)
lgb_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011826 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3347
[LightGBM] [Info] Number of data points in the train set: 33385, number of used features: 38
[LightGBM] [Info] Start training from score 13.507787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMRegressor(colsample_bytree=0.9223542176970055,
              learning_rate=0.09880695299818623, max_depth=14,
              n_estimators=1900, num_leaves=74, random_state=42,
              subsample=0.8042068321578268)

In [7]:
y_pred_lgb = lgb_model.predict(X_test)

In [10]:
mae = mean_absolute_error(y_test, y_pred_lgb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
print('mae:',mae)
print('rmse:',rmse)

mae: 0.0573831543762593
rmse: 0.09261737537582033
